In [1]:
!pip install -qq nest_asyncio pydantic pydantic_ai   python-dotenv 

In [1]:
import nest_asyncio 
nest_asyncio.apply()

In [2]:
from dotenv import load_dotenv
from pydantic_ai import Agent
from rich import print as pprint


In [3]:
load_dotenv()

agent = Agent('groq:llama-3.2-3b-preview', system_prompt='Be Descriptive.')

result = agent.run_sync('How does   orchestrator in ai agent run function .How is it allowed to do so' 
                        )
print(result.data)


An Orchestrator in an AI agent is a critical component that enables the agent to manage and execute multiple tasks, policies, or sub-agents efficiently and effectively. The Orchestrator is essentially a decision-making entity that coordinates with other components of the agent, allocating resources, and handling conflicts or dependencies between tasks.

Here's a step-by-step explanation of how the Orchestrator runs functions in an AI agent:

1. **Task Management**: The Orchestrator acts as a centralized task manager, responsible for identifying, prioritizing, and allocating tasks to different components of the agent.
2. **Task Instantiation**: When a new task is identified, the Orchestrator instantiates a new task entity, which includes relevant information, such as input parameters, constraints, and dependencies.
3. **Task Scheduling**: The Orchestrator schedules the task on the available resources, such as hardware or CPU time. It considers factors like task dependencies, resource al

In [40]:
from pydantic_ai import Agent, RunContext

roulette_agent = Agent(
    'groq:llama-3.2-3b-preview',
    deps_type=int,
    result_type=bool,
    system_prompt=(
        "Use the 'roulette_wheel' function to see if the "
        "customer has won based on the number they provided"
    )
)

@roulette_agent.tool
async def roulette_wheel(ctx:RunContext[int])

SyntaxError: expected ':' (1278878362.py, line 14)

In [93]:
from datetime import date

from pydantic_ai import Agent ,RunContext


agent = Agent(
    'groq:llama-3.2-3b-preview',
    deps_type=str,  #The agent expects a string dependency.
    system_prompt="Use the customer's name while replying to them"
)

@agent.system_prompt
def add_the_users_name(ctx:RunContext[str]) -> str:
    return f"The user name is {ctx.deps}"

@agent.system_prompt
def add_the_data() ->str:
    return f"The date is {date.today()}"

#they're appended in the order they're defined at runtime.

result = agent.run_sync('What is the date', deps="Frank")
print(result)
print(result.data)

RunResult(_all_messages=[ModelRequest(parts=[SystemPromptPart(content="Use the customer's name while replying to them", part_kind='system-prompt'), SystemPromptPart(content='The user name is Frank', part_kind='system-prompt'), SystemPromptPart(content='The date is 2024-12-30', part_kind='system-prompt'), UserPromptPart(content='What is the date', timestamp=datetime.datetime(2024, 12, 30, 18, 17, 29, 831408, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[TextPart(content='Happy New Year, Frank, \n\nThe current date is December 30th, 2024.', part_kind='text')], timestamp=datetime.datetime(2024, 12, 30, 18, 17, 35, tzinfo=datetime.timezone.utc), kind='response')], _new_message_index=0, data='Happy New Year, Frank, \n\nThe current date is December 30th, 2024.', _usage=Usage(requests=1, request_tokens=66, response_tokens=21, total_tokens=87, details=None))
Happy New Year, Frank, 

The current date is December 30th, 2024.


In [92]:
from pydantic import BaseModel

from pydantic_ai import Agent ,RunContext,ModelRetry,UnexpectedModelBehavior
from rich import print as pprint

class ChatResult(BaseModel):
    user_id:int
    message:str

agent =Agent(
    'groq:llama-3.1-70b-versatile',
    #result_type=ChatResult
)


@agent.tool(retries = 2)
def get_user_by_id(ctx:RunContext[int],name:str)->int:
    "Returns the provided id"
    user_id = ctx.deps; 

    if(user_id == 0):
        raise ModelRetry(
            "Please try again."
        )
    return user_id

try:
    result = agent.run_sync(
    "Use the provided tool and if id is 0 retry running the tool.Don't give random userid",deps=0)

except UnexpectedModelBehavior as e:
    print('An error occured:',e)
    print('cause:',repr(e.__cause__))
else:
    pprint(result.data)




<function=get_user_by_id({"name": "0"})</function>

In [44]:
from pydantic_ai import Agent,ModelRetry,UnexpectedModelBehavior
from rich import print as pprint
agent = Agent("groq:llama-3.1-70b-versatile",
              
              system_prompt="""You are an calcualtor which uses 
              the provided tools to perform calculation.Follow
              the instructions when i provided them and do as 
              said
              """
              )

@agent.tool_plain
def calc_volume(size: int) -> int:  
    if size == 42:
        return size**3
    else:
        raise ModelRetry('Please try again.')

try:
    result = agent.run_sync('please get me the volume of a box with size 3 with tool i provided')

except UnexpectedModelBehavior as e:
    print('An error occured:',e)
    print('cause:',repr(e.__cause__))
else:
    pprint(result)





An error occured: Tool exceeded max retries count of 1
cause: ModelRetry('Please try again.')


In [94]:
import random

from pydantic_ai import Agent,RunContext

@agent.tool_plain
def roll_die() -> str:
    "Roll a six-sided die and return the result"
    return str(random.randint)

@agent.tool
def get_player_name(ctx:RunContext[str])-> str:
    """Get the player's name"""  
    return ctx.deps


agent = Agent(
    "groq:llama-3.2-3b-preview",
    deps_type=str,
    system_prompt=(
        "You're a dice game , you should roll the die and see if the number"
        "you get back matches with user's guess . If so , tell them they are a winner"
        "Use the player's name in the response"
    )
)


dice_result = agent.run_sync("My guesss is 4", deps="Elnora")  


In [96]:
print(dice_result.data)
pprint(dice_result)


Let's roll the die... 

Rolling... 

The result is... **3**

Unfortunately, it doesn't match your guess. Better luck next time.

How about trying again?


RunResult(
    _all_messages=[
        ModelRequest(
            parts=[
                SystemPromptPart(
                    content="You're a dice game , you should roll the die and see if the numberyou get back matches
with user's guess . If so , tell them they are a winnerUse the player's name in the response",
                    part_kind='system-prompt'
                ),
                UserPromptPart(
                    content='My guesss is 4',
                    timestamp=datetime.datetime(2024, 12, 30, 18, 17, 55, 185199, tzinfo=datetime.timezone.utc),
                    part_kind='user-prompt'
                )
            ],
            kind='request'
        ),
        ModelResponse(
            parts=[
                TextPart(
                    content="Let's roll the die... \n\nRolling... \n\nThe result is... **3**\n\nUnfortunately, it 
doesn't match your guess. Better luck next time.\n\nHow about trying again?",
                    part_kind='text'
                )
            ],
            timestamp=datetime.datetime(2024, 12, 30, 18, 18, 1, tzinfo=datetime.timezone.utc),
            kind='response'
        )
    ],
    _new_message_index=0,
    data="Let's roll the die... \n\nRolling... \n\nThe result is... **3**\n\nUnfortunately, it doesn't match your 
guess. Better luck next time.\n\nHow about trying again?",
    _usage=Usage(requests=1, request_tokens=83, response_tokens=38, total_tokens=121, details=None)
)

In [9]:
!pip install rich

In [12]:
from rich import print as pprint

pprint(dice_result)

RunResult(
    _all_messages=[
        ModelRequest(
            parts=[
                SystemPromptPart(content='Be Descriptive.', part_kind='system-prompt'),
                UserPromptPart(
                    content='My guesss is 4',
                    timestamp=datetime.datetime(2024, 12, 30, 15, 11, 43, 306476, tzinfo=datetime.timezone.utc),
                    part_kind='user-prompt'
                )
            ],
            kind='request'
        ),
        ModelResponse(
            parts=[
                TextPart(
                    content="That's an interesting guess, but I'm not sure what we're referring to. Can you please 
provide more context or information about what you're trying to guess?",
                    part_kind='text'
                )
            ],
            timestamp=datetime.datetime(2024, 12, 30, 15, 11, 50, tzinfo=datetime.timezone.utc),
            kind='response'
        )
    ],
    _new_message_index=0,
    data="That's an interesting guess, but I'm not sure what we're referring to. Can you please provide more 
context or information about what you're trying to guess?",
    _usage=Usage(requests=1, request_tokens=45, response_tokens=34, total_tokens=79, details=None)
)

In [10]:
import random

from pydantic_ai import Agent,RunContext,Tool

def roll_die() -> str:
    "Roll a six-sided die and return the result"
    return str(random.randint)

def get_player_name(ctx:RunContext[str])-> str:
    """Get the player's name"""  
    return ctx.deps
dice_result = agent.run_sync("My guesss is 4", deps="Elnora")  

agent = Agent(
    "groq:llama-3.2-3b-preview",
    deps_type=str,
    system_prompt=(
        "You're a dice game , you should roll the die and see if the number"
        "you get back matches with user's guess . If so , tell them they are a winner"
        "Use the player's name in the response"
    ),

    tools = [
        Tool(roll_die,takes_ctx=False),
        Tool(get_player_name,takes_ctx=True)
    ]
)




In [16]:
pprint(dice_result.data)

You rolled a 2. Unfortunately, it doesn't match your guess, 4. Better luck next time, Elnora!

In [16]:
from pydantic_ai import Agent
from pydantic_ai.messages import ModelMessage,ModelResponse
from pydantic_ai.models.function import AgentInfo,FunctionModel

agent = Agent()

@agent.tool_plain
def foobar(a:int , b:str , c: dict[str, list[float]])->str:
    """Get me foobar.

    Args:
        a: apple pie
        b: banana cake
        c: carrot smoothie
    """
        
    return f'{a} {b} {c}'

def print_schema(messages:list[ModelMessage],info:AgentInfo)->ModelResponse:
    tool = info.function_tools[0]
    pprint(tool.description)
    pprint(tool.parameters_json_schema)

    return ModelResponse.from_text(content="foobar")


agent.run_sync('hello',model=FunctionModel(print_schema))

Get me foobar.

{
    'properties': {
        'a': {'description': 'apple pie', 'title': 'A', 'type': 'integer'},
        'b': {'description': 'banana cake', 'title': 'B', 'type': 'string'},
        'c': {
            'additionalProperties': {'items': {'type': 'number'}, 'type': 'array'},
            'description': 'carrot smoothie',
            'title': 'C',
            'type': 'object'
        }
    },
    'required': ['a', 'b', 'c'],
    'type': 'object',
    'additionalProperties': False
}

RunResult(_all_messages=[ModelRequest(parts=[UserPromptPart(content='hello', timestamp=datetime.datetime(2024, 12, 30, 15, 16, 32, 731989, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[TextPart(content='foobar', part_kind='text')], timestamp=datetime.datetime(2024, 12, 30, 15, 16, 32, 739322, tzinfo=datetime.timezone.utc), kind='response')], _new_message_index=0, data='foobar', _usage=Usage(requests=1, request_tokens=51, response_tokens=1, total_tokens=52, details=None))

In [63]:
from pydantic import BaseModel

from pydantic_ai import Agent
from pydantic_ai.models.test  import TestModel

agent =Agent()

class Foobar(BaseModel):
    "This is foobar"
    x:int
    y:str
    z:float = 3.14

@agent.tool_plain
def foobar(f :Foobar) ->str:
    return str(f)

test_model =TestModel()

result = agent.run_sync('hello',model=test_model)
pprint(result.data)
pprint(test_model.agent_model_function_tools)


{"foobar":"x=0 y='a' z=3.14"}

[
    ToolDefinition(
        name='foobar',
        description='This is foobar',
        parameters_json_schema={
            'properties': {
                'x': {'title': 'X', 'type': 'integer'},
                'y': {'title': 'Y', 'type': 'string'},
                'z': {'default': 3.14, 'title': 'Z', 'type': 'number'}
            },
            'required': ['x', 'y'],
            'title': 'Foobar',
            'type': 'object'
        },
        outer_typed_dict_key=None
    )
]

In [ ]:
from pydantic import BaseModel

class Foobar(BaseModel):
    "This is foobar"
    x:int
    y:str
    z:float = 3.14

def foobar(f :Foobar) ->str:
    return str(f)

In [24]:
from typing import Union

from pydantic_ai import Agent, RunContext
from pydantic_ai.tools import ToolDefinition

agent=Agent('test')

async def only_if_42(
        ctx:RunContext[int],
        tool_def:ToolDefinition
) -> Union[ToolDefinition,None]:
    if ctx.deps == 42:
        return tool_def

@agent.tool(prepare=only_if_42) # calling only_if_42 to check mayve
def hitchhiker(ctx:RunContext[int], answer:str) ->str:
    return f'{ctx.deps} {answer}'

result = agent.run_sync('testing ..',deps = 41)
pprint(result.data)
result = agent.run_sync('testing ..',deps = 42)
pprint(result.data)

success (no tool calls)

{"hitchhiker":"42 a"}

In [65]:
from __future__ import annotations

from typing import Literal

from pydantic_ai import Agent,RunContext
from pydantic_ai.models.test import TestModel
from pydantic_ai.tools import Tool, ToolDefinition

def greet(name:str)->str:
    return f'hello {name}'

async def prepare_greet(
        ctx:RunContext[Literal['human','machine']],
        tool_def:ToolDefinition,
) ->ToolDefinition | None:
    d= f'Name of the {ctx.deps} to greet'
    tool_def.parameters_json_schema['properties']['name']['description'] =d
    return tool_def

greet_tool = Tool(greet,prepare=prepare_greet)

test_model= TestModel()
agent = Agent(test_model,tools =[greet_tool], deps_type=Literal['human','machine'])

result = agent.run_sync('testing...',deps="human")
print(result.data)

pprint(test_model.agent_model_function_tools)


{"greet":"hello a"}


[
    ToolDefinition(
        name='greet',
        description='',
        parameters_json_schema={
            'properties': {
                'name': {'title': 'Name', 'type': 'string', 'description': 'Name of the human to greet'}
            },
            'required': ['name'],
            'type': 'object',
            'additionalProperties': False
        },
        outer_typed_dict_key=None
    )
]

In [ ]:
from typing import Callable

# Step 1: Define the 'prepare' function
def only_if_42(ctx: int) -> bool:
    """
    This function checks if the context is 42.
    If the context is 42, the function allows the tool to run.
    Otherwise, it prevents the tool from running.
    """
    return ctx == 42

# Step 2: Define the 'tool' decorator that accepts a 'prepare' argument
def tool(prepare: Callable[[int], bool]):
    def decorator(func: Callable):
        def wrapper(ctx: int, *args, **kwargs):
            # Step 3: Check if the prepare function allows execution
            if prepare(ctx):
                print(f"Executing {func.__name__} with context: {ctx}")
                return func(ctx, *args, **kwargs)
            else:
                print(f"Tool not applied because context ({ctx}) is not 42.")
                return None
        return wrapper
    return decorator

# Step 4: Apply the 'tool' decorator to a function
@tool(prepare=only_if_42)
def hitchhiker(ctx: int, answer: str) -> str:
    return f"The answer is {answer}, and the context is {ctx}"

# Step 5: Run the function with different contexts

# Test 1: When ctx is 41 (it should NOT execute)
print(hitchhiker(41, "Life"))

# Test 2: When ctx is 42 (it should execute)
print(hitchhiker(42, "Life"))


# Messages and chat history


In [66]:
from pydantic_ai import Agent

agent = Agent(
    "groq:llama-3.2-3b-preview",
    system_prompt="Be a helpful assistant."
)

result = agent.run_sync('Tell me a joke')
pprint(result)   # may be __repr__ metod does not include all messages method in it
pprint(result.data)

pprint(result.all_messages())



RunResult(
    _all_messages=[
        ModelRequest(
            parts=[
                SystemPromptPart(content='Be a helpful assistant.', part_kind='system-prompt'),
                UserPromptPart(
                    content='Tell me a joke',
                    timestamp=datetime.datetime(2024, 12, 30, 17, 28, 25, 603763, tzinfo=datetime.timezone.utc),
                    part_kind='user-prompt'
                )
            ],
            kind='request'
        ),
        ModelResponse(
            parts=[
                TextPart(
                    content="Here's one:\n\nWhy don't eggs tell jokes?\n\n(Wait for it...)\n\nBecause they'd crack 
each other up!",
                    part_kind='text'
                )
            ],
            timestamp=datetime.datetime(2024, 12, 30, 17, 28, 31, tzinfo=datetime.timezone.utc),
            kind='response'
        )
    ],
    _new_message_index=0,
    data="Here's one:\n\nWhy don't eggs tell jokes?\n\n(Wait for it...)\n\nBecause they'd crack each other up!",
    _usage=Usage(requests=1, request_tokens=44, response_tokens=25, total_tokens=69, details=None)
)

Here's one:

Why don't eggs tell jokes?

(Wait for it...)

Because they'd crack each other up!

[
    ModelRequest(
        parts=[
            SystemPromptPart(content='Be a helpful assistant.', part_kind='system-prompt'),
            UserPromptPart(
                content='Tell me a joke',
                timestamp=datetime.datetime(2024, 12, 30, 17, 28, 25, 603763, tzinfo=datetime.timezone.utc),
                part_kind='user-prompt'
            )
        ],
        kind='request'
    ),
    ModelResponse(
        parts=[
            TextPart(
                content="Here's one:\n\nWhy don't eggs tell jokes?\n\n(Wait for it...)\n\nBecause they'd crack each
other up!",
                part_kind='text'
            )
        ],
        timestamp=datetime.datetime(2024, 12, 30, 17, 28, 31, tzinfo=datetime.timezone.utc),
        kind='response'
    )
]

In [76]:
from pydantic_ai import Agent

agent =Agent(
    'groq:llama-3.2-3b-preview',
    system_prompt="Be a helpful assistant."
)

async def main():
    async with agent.run_stream('Tell me a quote in 5 words') as result:
        pprint(result.all_messages())
    
        async for text in result.stream():
            pprint(text)

        pprint(result.all_messages())

await main()


[
    ModelRequest(
        parts=[
            SystemPromptPart(content='Be a helpful assistant.', part_kind='system-prompt'),
            UserPromptPart(
                content='Tell me a quote in 5 words',
                timestamp=datetime.datetime(2024, 12, 30, 17, 47, 23, 729526, tzinfo=datetime.timezone.utc),
                part_kind='user-prompt'
            )
        ],
        kind='request'
    )
]

"Believe you can and achieve."

[
    ModelRequest(
        parts=[
            SystemPromptPart(content='Be a helpful assistant.', part_kind='system-prompt'),
            UserPromptPart(
                content='Tell me a quote in 5 words',
                timestamp=datetime.datetime(2024, 12, 30, 17, 47, 23, 729526, tzinfo=datetime.timezone.utc),
                part_kind='user-prompt'
            )
        ],
        kind='request'
    ),
    ModelResponse(
        parts=[TextPart(content='"Believe you can and achieve."', part_kind='text')],
        timestamp=datetime.datetime(2024, 12, 30, 17, 47, 24, 139728, tzinfo=datetime.timezone.utc),
        kind='response'
    )
]

In [78]:
from pydantic_ai import Agent

agent = Agent(
    'groq:llama-3.2-1b-preview',
    system_prompt="Be a helpful assistant"
)

result1=  agent.run_sync('tell me a joke.')
pprint(result1.data)

pprint(result1.new_messages())
result2 = agent.run_sync('Explain ' , message_history=result1.new_messages())

pprint(result2.all_messages())
pprint(result2.new_messages())



A man walked into a library and asked the librarian, "Do you have any books on Pavlov's dogs and Schrödinger's 
cat?"

The librarian replied, "It rings a bell, but I'm not sure if it's here or not."

[
    ModelRequest(
        parts=[
            SystemPromptPart(content='Be a helpful assistant', part_kind='system-prompt'),
            UserPromptPart(
                content='tell me a joke.',
                timestamp=datetime.datetime(2024, 12, 30, 18, 3, 29, 671737, tzinfo=datetime.timezone.utc),
                part_kind='user-prompt'
            )
        ],
        kind='request'
    ),
    ModelResponse(
        parts=[
            TextPart(
                content='A man walked into a library and asked the librarian, "Do you have any books on Pavlov\'s 
dogs and Schrödinger\'s cat?"\n\nThe librarian replied, "It rings a bell, but I\'m not sure if it\'s here or 
not."',
                part_kind='text'
            )
        ],
        timestamp=datetime.datetime(2024, 12, 30, 18, 3, 35, tzinfo=datetime.timezone.utc),
        kind='response'
    )
]

[
    ModelRequest(
        parts=[
            SystemPromptPart(content='Be a helpful assistant', part_kind='system-prompt'),
            UserPromptPart(
                content='tell me a joke.',
                timestamp=datetime.datetime(2024, 12, 30, 18, 3, 29, 671737, tzinfo=datetime.timezone.utc),
                part_kind='user-prompt'
            )
        ],
        kind='request'
    ),
    ModelResponse(
        parts=[
            TextPart(
                content='A man walked into a library and asked the librarian, "Do you have any books on Pavlov\'s 
dogs and Schrödinger\'s cat?"\n\nThe librarian replied, "It rings a bell, but I\'m not sure if it\'s here or 
not."',
                part_kind='text'
            )
        ],
        timestamp=datetime.datetime(2024, 12, 30, 18, 3, 35, tzinfo=datetime.timezone.utc),
        kind='response'
    ),
    ModelRequest(
        parts=[
            UserPromptPart(
                content='Explain ',
                timestamp=datetime.datetime(2024, 12, 30, 18, 3, 30, 111807, tzinfo=datetime.timezone.utc),
                part_kind='user-prompt'
            )
        ],
        kind='request'
    ),
    ModelResponse(
        parts=[
            TextPart(
                content='This joke plays on two scientific concepts: Pavlov\'s dogs, a classic experiment where 
Ivan Pavlov showed that dogs (and humans) can become conditioned to associate a neutral stimulus with a stressful 
or neutral one, leading to an automatic response.\n\nSchrödinger\'s cat was a thought experiment created by Erwin 
Schrödinger, a physicist, where he said that a cat is both alive AND dead at the same time until an observer 
catches it. This is because the act of observation itself can "reduce" the superposition of states and cause the 
cat\'s state to be definitively decided.\n\nIn the joke, the librarian asks if the man has a book on both Pavlov\'s
dogs and Schrödinger\'s cat. "It rings a bell," she says, a classic reference to Pavlov\'s dogs. However, "I\'m not
sure if it\'s here or not." \n\nThe word "bell" is a clever play on words. In this context, it means that the 
question is being asked if the book exists (being "rings a bell"), but it also references the Russian physicist for
the second part of the sentence, which could be interpreted as saying that the book doesn\'t exist (being "not not"
or "un exist").\n\nThe punchline is that the librarian\'s response somehow connects the concept of Pavlov\'s dogs 
with Schrödinger\'s cat in a peculiar way, creating a humorous and unexpected connection between the two.',
                part_kind='text'
            )
        ],
        timestamp=datetime.datetime(2024, 12, 30, 18, 3, 35, tzinfo=datetime.timezone.utc),
        kind='response'
    )
]

[
    ModelRequest(
        parts=[
            UserPromptPart(
                content='Explain ',
                timestamp=datetime.datetime(2024, 12, 30, 18, 3, 30, 111807, tzinfo=datetime.timezone.utc),
                part_kind='user-prompt'
            )
        ],
        kind='request'
    ),
    ModelResponse(
        parts=[
            TextPart(
                content='This joke plays on two scientific concepts: Pavlov\'s dogs, a classic experiment where 
Ivan Pavlov showed that dogs (and humans) can become conditioned to associate a neutral stimulus with a stressful 
or neutral one, leading to an automatic response.\n\nSchrödinger\'s cat was a thought experiment created by Erwin 
Schrödinger, a physicist, where he said that a cat is both alive AND dead at the same time until an observer 
catches it. This is because the act of observation itself can "reduce" the superposition of states and cause the 
cat\'s state to be definitively decided.\n\nIn the joke, the librarian asks if the man has a book on both Pavlov\'s
dogs and Schrödinger\'s cat. "It rings a bell," she says, a classic reference to Pavlov\'s dogs. However, "I\'m not
sure if it\'s here or not." \n\nThe word "bell" is a clever play on words. In this context, it means that the 
question is being asked if the book exists (being "rings a bell"), but it also references the Russian physicist for
the second part of the sentence, which could be interpreted as saying that the book doesn\'t exist (being "not not"
or "un exist").\n\nThe punchline is that the librarian\'s response somehow connects the concept of Pavlov\'s dogs 
with Schrödinger\'s cat in a peculiar way, creating a humorous and unexpected connection between the two.',
                part_kind='text'
            )
        ],
        timestamp=datetime.datetime(2024, 12, 30, 18, 3, 35, tzinfo=datetime.timezone.utc),
        kind='response'
    )
]

In [79]:
from pydantic_ai import Agent

agent = Agent(
    'groq:llama-3.2-1b-preview',
)

result = agent.run_sync("Hello")
pprint(result.all_messages())

[
    ModelRequest(
        parts=[
            UserPromptPart(
                content='Hello',
                timestamp=datetime.datetime(2024, 12, 30, 18, 6, 45, 207481, tzinfo=datetime.timezone.utc),
                part_kind='user-prompt'
            )
        ],
        kind='request'
    ),
    ModelResponse(
        parts=[
            TextPart(
                content='Hello! Is there something I can help you with or would you like to chat today?',
                part_kind='text'
            )
        ],
        timestamp=datetime.datetime(2024, 12, 30, 18, 6, 51, tzinfo=datetime.timezone.utc),
        kind='response'
    )
]

In [98]:
from pydantic import BaseModel

from pydantic_ai import Agent

class CityLocation(BaseModel):
    city:str
    country:str

agent = Agent('groq:llama-3.3-70b-versatile',result_type=CityLocation)

result = agent.run_sync('Where were the olympics held in 2024')

pprint(result.data)

pprint(result.usage)

CityLocation(city='Paris', country='France')

<bound method RunResult.usage of RunResult(_all_messages=[ModelRequest(parts=[UserPromptPart(content='Where were 
the olympics held in 2024', timestamp=datetime.datetime(2024, 12, 30, 18, 21, 41, 543808, 
tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), 
ModelResponse(parts=[ToolCallPart(tool_name='final_result', args=ArgsJson(args_json='{"city": "Paris", "country": 
"France"}'), tool_call_id='call_8sxf', part_kind='tool-call')], timestamp=datetime.datetime(2024, 12, 30, 18, 21, 
49, tzinfo=datetime.timezone.utc), kind='response'), ModelRequest(parts=[ToolReturnPart(tool_name='final_result', 
content='Final result processed.', tool_call_id='call_8sxf', timestamp=datetime.datetime(2024, 12, 30, 18, 21, 43, 
956242, tzinfo=datetime.timezone.utc), part_kind='tool-return')], kind='request')], _new_message_index=0, 
data=CityLocation(city='Paris', country='France'), _usage=Usage(requests=1, request_tokens=525, response_tokens=18,
total_tokens=543, details=None))>

In [99]:
pprint(result)

RunResult(
    _all_messages=[
        ModelRequest(
            parts=[
                UserPromptPart(
                    content='Where were the olympics held in 2024',
                    timestamp=datetime.datetime(2024, 12, 30, 18, 21, 41, 543808, tzinfo=datetime.timezone.utc),
                    part_kind='user-prompt'
                )
            ],
            kind='request'
        ),
        ModelResponse(
            parts=[
                ToolCallPart(
                    tool_name='final_result',
                    args=ArgsJson(args_json='{"city": "Paris", "country": "France"}'),
                    tool_call_id='call_8sxf',
                    part_kind='tool-call'
                )
            ],
            timestamp=datetime.datetime(2024, 12, 30, 18, 21, 49, tzinfo=datetime.timezone.utc),
            kind='response'
        ),
        ModelRequest(
            parts=[
                ToolReturnPart(
                    tool_name='final_result',
                    content='Final result processed.',
                    tool_call_id='call_8sxf',
                    timestamp=datetime.datetime(2024, 12, 30, 18, 21, 43, 956242, tzinfo=datetime.timezone.utc),
                    part_kind='tool-return'
                )
            ],
            kind='request'
        )
    ],
    _new_message_index=0,
    data=CityLocation(city='Paris', country='France'),
    _usage=Usage(requests=1, request_tokens=525, response_tokens=18, total_tokens=543, details=None)
)

In [100]:
pprint(result.all_messages())  # need tp understand 
pprint(result.data)

[
    ModelRequest(
        parts=[
            UserPromptPart(
                content='Where were the olympics held in 2024',
                timestamp=datetime.datetime(2024, 12, 30, 18, 21, 41, 543808, tzinfo=datetime.timezone.utc),
                part_kind='user-prompt'
            )
        ],
        kind='request'
    ),
    ModelResponse(
        parts=[
            ToolCallPart(
                tool_name='final_result',
                args=ArgsJson(args_json='{"city": "Paris", "country": "France"}'),
                tool_call_id='call_8sxf',
                part_kind='tool-call'
            )
        ],
        timestamp=datetime.datetime(2024, 12, 30, 18, 21, 49, tzinfo=datetime.timezone.utc),
        kind='response'
    ),
    ModelRequest(
        parts=[
            ToolReturnPart(
                tool_name='final_result',
                content='Final result processed.',
                tool_call_id='call_8sxf',
                timestamp=datetime.datetime(2024, 12, 30, 18, 21, 43, 956242, tzinfo=datetime.timezone.utc),
                part_kind='tool-return'
            )
        ],
        kind='request'
    )
]

CityLocation(city='Paris', country='France')

In [21]:
load_dotenv()

True

In [26]:
from typing import Union

from pydantic import BaseModel

from pydantic_ai import Agent

class Box(BaseModel):
    width:int
    height:int
    depth:int
    units:str

agent: Agent[None, Union[Box,str]] = Agent(
    'gemini-1.5-flash',
    result_type=Union[Box,str],
    system_prompt=(
        "Dont assume any data and ignore the result format if you don't have all data"
        "Extract me the dimension of the box"
        "if you can't extract all data , ask the user try again."
    )
)


result = agent.run_sync('The box is 10x20x30')
pprint(result.data)
pprint(result.all_messages())

result = agent.run_sync("The box is 10x20x30 cm")

pprint(result.data)

pprint(result.all_messages())


I need more information to determine the units.  Please specify the units of measurement (e.g., inches, 
centimeters, meters).

[
    ModelRequest(
        parts=[
            SystemPromptPart(
                content="Dont assume any data and ignore the result format if you don't have all dataExtract me the
dimension of the boxif you can't extract all data , ask the user try again.",
                part_kind='system-prompt'
            ),
            UserPromptPart(
                content='The box is 10x20x30',
                timestamp=datetime.datetime(2024, 12, 31, 3, 55, 11, 297482, tzinfo=datetime.timezone.utc),
                part_kind='user-prompt'
            )
        ],
        kind='request'
    ),
    ModelResponse(
        parts=[
            TextPart(
                content='I need more information to determine the units.  Please specify the units of measurement 
(e.g., inches, centimeters, meters).\n',
                part_kind='text'
            )
        ],
        timestamp=datetime.datetime(2024, 12, 31, 3, 55, 13, 147883, tzinfo=datetime.timezone.utc),
        kind='response'
    )
]

Box(width=10, height=20, depth=30, units='cm')

[
    ModelRequest(
        parts=[
            SystemPromptPart(
                content="Dont assume any data and ignore the result format if you don't have all dataExtract me the
dimension of the boxif you can't extract all data , ask the user try again.",
                part_kind='system-prompt'
            ),
            UserPromptPart(
                content='The box is 10x20x30 cm',
                timestamp=datetime.datetime(2024, 12, 31, 3, 55, 13, 159116, tzinfo=datetime.timezone.utc),
                part_kind='user-prompt'
            )
        ],
        kind='request'
    ),
    ModelResponse(
        parts=[
            ToolCallPart(
                tool_name='final_result',
                args=ArgsDict(args_dict={'height': 20, 'depth': 30, 'width': 10, 'units': 'cm'}),
                tool_call_id=None,
                part_kind='tool-call'
            )
        ],
        timestamp=datetime.datetime(2024, 12, 31, 3, 55, 13, 966554, tzinfo=datetime.timezone.utc),
        kind='response'
    ),
    ModelRequest(
        parts=[
            ToolReturnPart(
                tool_name='final_result',
                content='Final result processed.',
                tool_call_id=None,
                timestamp=datetime.datetime(2024, 12, 31, 3, 55, 13, 968569, tzinfo=datetime.timezone.utc),
                part_kind='tool-return'
            )
        ],
        kind='request'
    )
]

In [5]:
from typing import Union

from pydantic import BaseModel

from pydantic_ai import Agent



agent= Agent(
    'gemini-1.5-flash',
    result_type=int,
    system_prompt=(
      "You are a calculator and you can only use the tool i provide."
    )
)

@agent.tool_plain
def additon( a:int , b:int)->int:
    "Perfoms addition"
    return a+b


result = agent.run_sync('What is 2 + 4')
pprint(result.data)
pprint(result.all_messages())


6

[
    ModelRequest(
        parts=[
            SystemPromptPart(
                content='You are a calculator and you can only use the tool i provide.',
                part_kind='system-prompt'
            ),
            UserPromptPart(
                content='What is 2 + 4',
                timestamp=datetime.datetime(2024, 12, 31, 5, 41, 10, 262703, tzinfo=datetime.timezone.utc),
                part_kind='user-prompt'
            )
        ],
        kind='request'
    ),
    ModelResponse(
        parts=[
            ToolCallPart(
                tool_name='additon',
                args=ArgsDict(args_dict={'b': 4, 'a': 2}),
                tool_call_id=None,
                part_kind='tool-call'
            )
        ],
        timestamp=datetime.datetime(2024, 12, 31, 5, 41, 12, 144381, tzinfo=datetime.timezone.utc),
        kind='response'
    ),
    ModelRequest(
        parts=[
            ToolReturnPart(
                tool_name='additon',
                content=6,
                tool_call_id=None,
                timestamp=datetime.datetime(2024, 12, 31, 5, 41, 12, 146023, tzinfo=datetime.timezone.utc),
                part_kind='tool-return'
            )
        ],
        kind='request'
    ),
    ModelResponse(
        parts=[
            ToolCallPart(
                tool_name='final_result',
                args=ArgsDict(args_dict={'response': 6}),
                tool_call_id=None,
                part_kind='tool-call'
            )
        ],
        timestamp=datetime.datetime(2024, 12, 31, 5, 41, 12, 838536, tzinfo=datetime.timezone.utc),
        kind='response'
    ),
    ModelRequest(
        parts=[
            ToolReturnPart(
                tool_name='final_result',
                content='Final result processed.',
                tool_call_id=None,
                timestamp=datetime.datetime(2024, 12, 31, 5, 41, 12, 839550, tzinfo=datetime.timezone.utc),
                part_kind='tool-return'
            )
        ],
        kind='request'
    )
]

In [25]:
from typing import Union 

from pydantic_ai import Agent

agent:Agent[None,Union[list[str],list[int]]] =Agent(
    'groq:llama-3.1-70b-versatile',
    result_type=Union[list[str],list[int]],
    system_prompt="Extract either colors or sizes as list from the shapes provided"
)


result = agent.run_sync('red square ,blue circle,green rectangle')
pprint(result.data)
pprint(result.all_messages())


result = agent.run_sync('square size 10,circle size 20,triangle size 30')
pprint(result.data)



['red', 'blue', 'green']

[
    ModelRequest(
        parts=[
            SystemPromptPart(
                content='Extract either colors or sizes as list from the shapes provided',
                part_kind='system-prompt'
            ),
            UserPromptPart(
                content='red square ,blue circle,green rectangle',
                timestamp=datetime.datetime(2024, 12, 31, 4, 59, 55, 548097, tzinfo=datetime.timezone.utc),
                part_kind='user-prompt'
            )
        ],
        kind='request'
    ),
    ModelResponse(
        parts=[
            ToolCallPart(
                tool_name='final_result_list',
                args=ArgsJson(args_json='{"response": ["red", "blue", "green"]}'),
                tool_call_id='call_vmfg',
                part_kind='tool-call'
            )
        ],
        timestamp=datetime.datetime(2024, 12, 31, 5, 0, 1, tzinfo=datetime.timezone.utc),
        kind='response'
    ),
    ModelRequest(
        parts=[
            ToolReturnPart(
                tool_name='final_result_list',
                content='Final result processed.',
                tool_call_id='call_vmfg',
                timestamp=datetime.datetime(2024, 12, 31, 4, 59, 56, 25694, tzinfo=datetime.timezone.utc),
                part_kind='tool-return'
            )
        ],
        kind='request'
    )
]

[10, 20, 30]

https://github.com/mistralai/cookbook/blob/main/third_party/PydanticAI/pydantic_bank_support_agent.ipynb
https://github.com/pydantic/pydantic-ai/blob/main/tests/example_modules/fake_database.py

In [18]:
from typing import Union
from fake_database import DatabaseConn,QueryError

from pydantic import BaseModel

from pydantic_ai import Agent, RunContext,ModelRetry

class Success(BaseModel):
    sql_query:str

class InvalidRequest(BaseModel):
    error_message:str

Response = Union[Success,InvalidRequest]

agent :Agent[DatabaseConn,Response] =Agent(
    "gemini-1.5-flash",
    result_type=Response,
    deps_type=DatabaseConn,
    system_prompt="Generate PostgreSQL flavored SQL queries based on user input"
)


@agent.result_validator
async def validate_result(ctx:RunContext[DatabaseConn],result:Response)->Response:
    if isinstance(result,InvalidRequest):
        return result
    try:
        await ctx.deps.execute(f'Explain {result.sql_query}')
    
    except QueryError:
        raise ModelRetry(f'Invalid query: {e}') from e
    else:
        return result
    

# result = agent.run_sync(
#     'get me puppies who were dancing', deps =DatabaseConn()
# )



# pprint(result.data)


result = agent.run_sync(
    'get me uses who were last active yesterday.', deps =DatabaseConn()
)

pprint(result.data)
pprint(result.all_messages())


Success(sql_query='\\"SELECT * FROM users WHERE last_active_date = CURRENT_DATE - INTERVAL \'1 day\';\\"')

[
    ModelRequest(
        parts=[
            SystemPromptPart(
                content='Generate PostgreSQL flavored SQL queries based on user input',
                part_kind='system-prompt'
            ),
            UserPromptPart(
                content='get me uses who were last active yesterday.',
                timestamp=datetime.datetime(2024, 12, 31, 6, 8, 57, 37558, tzinfo=datetime.timezone.utc),
                part_kind='user-prompt'
            )
        ],
        kind='request'
    ),
    ModelResponse(
        parts=[
            ToolCallPart(
                tool_name='final_result_Success',
                args=ArgsDict(
                    args_dict={
                        'sql_query': '\\"SELECT * FROM users WHERE last_active_date = CURRENT_DATE - INTERVAL \'1 
day\';\\"'
                    }
                ),
                tool_call_id=None,
                part_kind='tool-call'
            )
        ],
        timestamp=datetime.datetime(2024, 12, 31, 6, 8, 59, 48476, tzinfo=datetime.timezone.utc),
        kind='response'
    ),
    ModelRequest(
        parts=[
            ToolReturnPart(
                tool_name='final_result_Success',
                content='Final result processed.',
                tool_call_id=None,
                timestamp=datetime.datetime(2024, 12, 31, 6, 8, 59, 48981, tzinfo=datetime.timezone.utc),
                part_kind='tool-return'
            )
        ],
        kind='request'
    )
]

In [22]:
from pydantic_ai import Agent

agent = Agent('gemini-1.5-flash')

async def main():
    async with agent.run_stream("Where does 'hello world' come from") as result:
        async for message in result.stream_text(delta=True):
            print(message)


await main()

The
 "Hello, world!" program originates from Brian Kernighan's 1
972 internal document, "A Tutorial Introduction to the Language B".  
While not the very first example of a simple program printing text, it's the one that popularized the phrase and the practice of using it as a beginner's programming exercise.  Its simplicity and widespread use cemented its place in programming history.  Before Kernighan's document, similar examples existed, but
 his version is the one that became the standard.



The final result message will NOT be added to result messages if you use .stream_text(delta=True), see Messages and chat history for more information.



In [44]:
from datetime import date

from typing_extensions import TypedDict

from pydantic_ai import Agent

class UserProfile(TypedDict,total=False):
    """
       name : Name of person
       dob :Date of birth of person
       bio :More information about the person
    """
    name:str
    dob:date
    bio:str


agent =Agent(
    "groq:llama-3.1-70b-versatile",
    result_type=UserProfile,
    system_prompt="Extract a user profile from the input"
)


async def main():
    user_input = 'Name is Ben , Born on January 28th 1990 and likes llms and AI '
    async with agent.run_stream(user_input) as result:
        async for profile in result.stream():
            print(profile)
        
        pprint(result.all_messages())



await main()

{'name': 'Ben', 'dob': datetime.date(1990, 1, 28), 'bio': 'likes llms and AI'}
{'name': 'Ben', 'dob': datetime.date(1990, 1, 28), 'bio': 'likes llms and AI'}


[
    ModelRequest(
        parts=[
            SystemPromptPart(content='Extract a user profile from the input', part_kind='system-prompt'),
            UserPromptPart(
                content='Name is Ben , Born on January 28th 1990 and likes llms and AI ',
                timestamp=datetime.datetime(2024, 12, 31, 7, 19, 5, 177981, tzinfo=datetime.timezone.utc),
                part_kind='user-prompt'
            )
        ],
        kind='request'
    ),
    ModelResponse(
        parts=[
            ToolCallPart(
                tool_name='final_result',
                args=ArgsJson(args_json='{"name": "Ben", "dob": "1990-01-28", "bio": "likes llms and AI"}'),
                tool_call_id='call_g1pp',
                part_kind='tool-call'
            )
        ],
        timestamp=datetime.datetime(2024, 12, 31, 7, 19, 11, tzinfo=datetime.timezone.utc),
        kind='response'
    ),
    ModelRequest(
        parts=[
            ToolReturnPart(
                tool_name='final_result',
                content='Final result processed.',
                tool_call_id='call_g1pp',
                timestamp=datetime.datetime(2024, 12, 31, 7, 19, 5, 719041, tzinfo=datetime.timezone.utc),
                part_kind='tool-return'
            )
        ],
        kind='request'
    )
]

In [ ]:
from datetime import date

from typing_extensions import TypedDict
from pydantic import ValidationError
from pydantic_ai import Agent

class UserProfile(TypedDict,total=False):
    """
       name : Name of person
       dob :Date of birth of person
       bio :More information about the person
    """
    name:str
    dob:date
    bio:str


agent =Agent(
    "groq:llama-3.2-3b-preview",
    result_type=UserProfile,
    system_prompt="Extract a user profile from the input"
)


async def main():
    user_input = 'Name is Ben , Born on January 28th 1990 and likes llms and AI '
    async with agent.run_stream(user_input) as result:
        async for message, last in result.stream_structured(debounce_by=0.01):
            try:
                profile = await result.validate_structured_result(
                    message,
                    allow_partial=not last
                )
            except ValidationError:
                continue
            print(profile)


await main()

[ModelRequest(parts=[SystemPromptPart(content='Extract a user profile from the input', part_kind='system-prompt'), UserPromptPart(content='Name is Ben , Born on January 28th 1990 and likes llms and AI ', timestamp=datetime.datetime(2024, 12, 31, 7, 17, 32, 326587, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[ToolCallPart(tool_name='final_result', args=ArgsJson(args_json='{"name": "Ben","dob": "January 28th 1990","bio": "likes llms and AI"}'), tool_call_id='call_98j4', part_kind='tool-call')], timestamp=datetime.datetime(2024, 12, 31, 7, 17, 39, tzinfo=datetime.timezone.utc), kind='response'), ModelRequest(parts=[ToolReturnPart(tool_name='final_result', content='Final result processed.', tool_call_id='call_98j4', timestamp=datetime.datetime(2024, 12, 31, 7, 17, 32, 776439, tzinfo=datetime.timezone.utc), part_kind='tool-return')], kind='request')]


# Dependencies


In [45]:
from dataclasses import dataclass

import httpx

from pydantic_ai import Agent


@dataclass
class MyDeps:
    api_key:str
    http_client: httpx.AsyncClient

agent = Agent(
    'gemini-1.5-flash',
    deps_type=MyDeps
)


async def main():
    async with httpx.AsyncClient() as client:
        deps= MyDeps('foobar',client)
        result = await agent.run('Tell me a joke.',deps = deps)

    print(result.data)

await main()

Why don't scientists trust atoms? 

Because they make up everything!



In [7]:
from dataclasses import dataclass

import httpx

from pydantic_ai import Agent, RunContext

@dataclass
class MyDeps:
    api_key:str
    http_client:httpx.AsyncClient

agent = Agent(
    "gemini-1.5-flash",
    deps_type=MyDeps
)


@agent.system_prompt
async def get_system_prompt(ctx:RunContext[MyDeps])-> str:
    response = await ctx.deps.http_client.get(
        'http://example.com',
        headers={'Authorization': f'Bearer {ctx.deps.api_key}'}
    )

    response.raise_for_status()
    return f'Prompt: {response.text}' 

async def main():
    async with httpx.AsyncClient() as client:
        deps = MyDeps('foobar',client)
        result = await agent.run('Tell me a joke',deps = deps)
        print(result.data)
        pprint(result.all_messages())



await main()



Why don't scientists trust atoms? 

Because they make up everything!



[
    ModelRequest(
        parts=[
            SystemPromptPart(
                content='Prompt: <!doctype html>\n<html>\n<head>\n    <title>Example Domain</title>\n\n    <meta 
charset="utf-8" />\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />\n    <meta 
name="viewport" content="width=device-width, initial-scale=1" />\n    <style type="text/css">\n    body {\n        
background-color: #f0f0f2;\n        margin: 0;\n        padding: 0;\n        font-family: -apple-system, system-ui,
BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;\n        \n    }\n    
div {\n        width: 600px;\n        margin: 5em auto;\n        padding: 2em;\n        background-color: 
#fdfdff;\n        border-radius: 0.5em;\n        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);\n    }\n    a:link, 
a:visited {\n        color: #38488f;\n        text-decoration: none;\n    }\n    @media (max-width: 700px) {\n     
div {\n            margin: 0 auto;\n            width: auto;\n        }\n    }\n    </style>    
\n</head>\n\n<body>\n<div>\n    <h1>Example Domain</h1>\n    <p>This domain is for use in illustrative examples in 
documents. You may use this\n    domain in literature without prior coordination or asking for permission.</p>\n   
<p><a href="https://www.iana.org/domains/example">More information...</a></p>\n</div>\n</body>\n</html>\n',
                part_kind='system-prompt'
            ),
            UserPromptPart(
                content='Tell me a joke',
                timestamp=datetime.datetime(2024, 12, 31, 11, 46, 33, 422231, tzinfo=datetime.timezone.utc),
                part_kind='user-prompt'
            )
        ],
        kind='request'
    ),
    ModelResponse(
        parts=[
            TextPart(
                content="Why don't scientists trust atoms? \n\nBecause they make up everything!\n",
                part_kind='text'
            )
        ],
        timestamp=datetime.datetime(2024, 12, 31, 11, 46, 35, 70138, tzinfo=datetime.timezone.utc),
        kind='response'
    )
]

In [27]:
from dataclasses import dataclass

import httpx

from pydantic_ai import Agent, RunContext

@dataclass
class MyDeps:
    api_key:str
    http_client:httpx.AsyncClient  # how is it working with sync??

agent = Agent(
    "gemini-1.5-flash",
    deps_type=MyDeps
)


@agent.system_prompt
def get_system_prompt(ctx:RunContext[MyDeps])-> str:
    response = ctx.deps.http_client.get(
        'http://example.com',
        headers={'Authorization': f'Bearer {ctx.deps.api_key}'}
    )

    response.raise_for_status()
    return f'Prompt: {response.text}' 

async def main():
        deps = MyDeps('foobar',httpx.Client())
        result = await agent.run('Tell me a joke',deps = deps)
        print(result.data)



await main()



Why don't scientists trust atoms? 

Because they make up everything!



In [23]:
from dataclasses import dataclass

import httpx

from pydantic_ai import Agent ,RunContext


@dataclass
class MyDeps:
    api_key:str
    http_client:httpx.AsyncClient


agent = Agent(
    'groq:llama-3.3-70b-versatile',
    deps_type=MyDeps
)

@agent.system_prompt
async def get_system_prompt(ctx=RunContext[MyDeps])->str:
    response = await ctx.deps.http_client.get('https://example.com')
    response.raise_for_status()
    return f'Prompt: {response.text}'

@agent.tool
async def get_joke_material(ctx:RunContext[MyDeps],subject:str)->str:
    response = await ctx.deps.http_client.get(
        'https://example.com#jokes',
        params ={"subject":subject},
        headers= {"Authorization": f'Bearer{ ctx.deps.api_key}'}
    )

    response.raise_for_status()
    return  response.text



@agent.result_validator
async def validate_result(ctx:RunContext[MyDeps],final_response:str)->str:
    response= await ctx.deps.http_client.get(
        'https://example.com#validate',
        headers= {'Authorization': f'Bearer: {ctx.deps.api_key}'},
        params = {'query':final_response}
    )

    if response.status_code==400:
        raise ModelRetry(f'invalid response: {response.text}')
    response.raise_for_status()
    return final_response


async def main():
    async with httpx.AsyncClient() as client:
        deps = MyDeps('foobar',client)
        result = await agent.run('Tell me a joke', deps = deps)
        print(result.data)
        pprint(result.all_messages())


await main()




<function=get_joke_material{"subject": "joke"}</function>


[
    ModelRequest(
        parts=[
            SystemPromptPart(
                content='Prompt: <!doctype html>\n<html>\n<head>\n    <title>Example Domain</title>\n\n    <meta 
charset="utf-8" />\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />\n    <meta 
name="viewport" content="width=device-width, initial-scale=1" />\n    <style type="text/css">\n    body {\n        
background-color: #f0f0f2;\n        margin: 0;\n        padding: 0;\n        font-family: -apple-system, system-ui,
BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;\n        \n    }\n    
div {\n        width: 600px;\n        margin: 5em auto;\n        padding: 2em;\n        background-color: 
#fdfdff;\n        border-radius: 0.5em;\n        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);\n    }\n    a:link, 
a:visited {\n        color: #38488f;\n        text-decoration: none;\n    }\n    @media (max-width: 700px) {\n     
div {\n            margin: 0 auto;\n            width: auto;\n        }\n    }\n    </style>    
\n</head>\n\n<body>\n<div>\n    <h1>Example Domain</h1>\n    <p>This domain is for use in illustrative examples in 
documents. You may use this\n    domain in literature without prior coordination or asking for permission.</p>\n   
<p><a href="https://www.iana.org/domains/example">More information...</a></p>\n</div>\n</body>\n</html>\n',
                part_kind='system-prompt'
            ),
            UserPromptPart(
                content='Tell me a joke',
                timestamp=datetime.datetime(2024, 12, 31, 13, 33, 54, 685035, tzinfo=datetime.timezone.utc),
                part_kind='user-prompt'
            )
        ],
        kind='request'
    ),
    ModelResponse(
        parts=[TextPart(content='<function=get_joke_material{"subject": "joke"}</function>', part_kind='text')],
        timestamp=datetime.datetime(2024, 12, 31, 13, 34, 1, tzinfo=datetime.timezone.utc),
        kind='response'
    )
]

# Problem in understanding

In [16]:
from dataclasses import dataclass

import httpx

from pydantic_ai import Agent ,RunContext


@dataclass
class MyDeps:
    api_key:str
    http_client:httpx.AsyncClient
    
    async def system_prompt_factory(self)->str:
        response = await self.http_client.get('https://example.com')
        response.raise_for_status()
        return f'Prompt: {response.text}'



agent = Agent(
    'groq:llama-3.3-70b-versatile',
    deps_type=MyDeps
)



joke_agent= Agent('gemini-1.5-flash',deps_type=MyDeps)

@joke_agent.system_prompt
async def get_system_prompt(ctx:RunContext[MyDeps])->str:
    return await ctx.deps.system_prompt_factory()

async def application_code(prompt:str)->str:
    # application code 

    async with httpx.AsyncClient() as client:
        app_deps = MyDeps('foobar',client)
        result = await joke_agent.run(prompt,deps = app_deps)
    return result.all_messages()



In [15]:
class TestMyDeps(MyDeps):
    async def system_prompt_factory(self):
        return 'test prompt'

async def test_application_code():
    test_deps = TestMyDeps('test_key',None)

    with joke_agent.override(deps =test_deps):
        joke = await application_code('Tell me a joke')
        pprint(joke)

await test_application_code()

[
    ModelRequest(
        parts=[
            SystemPromptPart(content='test prompt', part_kind='system-prompt'),
            UserPromptPart(
                content='Tell me a joke',
                timestamp=datetime.datetime(2024, 12, 31, 15, 26, 7, 164314, tzinfo=datetime.timezone.utc),
                part_kind='user-prompt'
            )
        ],
        kind='request'
    ),
    ModelResponse(
        parts=[
            TextPart(
                content="Why don't scientists trust atoms? \n\nBecause they make up everything!\n",
                part_kind='text'
            )
        ],
        timestamp=datetime.datetime(2024, 12, 31, 15, 26, 7, 842093, tzinfo=datetime.timezone.utc),
        kind='response'
    )
]

In [55]:
from dataclasses import dataclass

from pydantic_ai import Agent,RunContext

@dataclass
class MyDeps:
    factory_agent: Agent[None,list[str]]


@dataclass
class Jokes:
    "jokes: a list of strings containing jokes"
    "best joke:a joke selected among them"
    jokes:list[str]
    bestJoke:str

joke_agent= Agent(
    'groq:llama-3.1-70b-versatile',
    deps_type=MyDeps,
    system_prompt=(
        'use the joke factory to generate some jokes, then choose the best'
    ),
    result_type=Jokes
)


factory_agent= Agent('ollama:llama3.2',result_type=list[str])


@joke_agent.tool
async def joke_factory(ctx:RunContext[MyDeps], count:int)->str:
    r= await ctx.deps.factory_agent.run('Please generate {count} jokes.')
    return '\n'.join(r.data)

result =joke_agent.run_sync('Tell me a joke', deps=MyDeps(factory_agent))
pprint(result.data)


Jokes(jokes=['joke1', 'joke2', 'joke3'], bestJoke='joke1')

In [58]:
from pydantic_ai import Agent ,RunContext

roulette_agent = Agent(
    'gemini-1.5-flash',
    deps_type=int,
    result_type=bool,
    system_prompt=(
        "Use the roulette_wheel function to see if "
        "customer has won based on the number they provide"
    )
)


@roulette_agent.tool
async def roulette_wheel(ctx:RunContext[int], square:int)->str:
    'check if the square is a winner'
    return 'winner' if square == ctx.deps else 'loser'

success_number = 18

result =roulette_agent.run_sync('Player chose 17', deps = success_number)
pprint(result.data)

False

In [61]:
from pydantic_ai import Agent

agent = Agent('gemini-1.5-flash')

result_sync = agent.run_sync('What is the capital of italy?')
print(result_sync.data)

async def main():
    result = await agent.run("What is the capital of France?")
    print(result.data)

    async with agent.run_stream('What is the capital of USA?') as response:
        print(await response.get_data())

await main()


Rome

Paris

The capital of the USA is **Washington, D.C.**



In [63]:
from pydantic_ai import Agent
from pydantic_ai.exceptions import UsageLimitExceeded
from pydantic_ai.settings import UsageLimits


agent = Agent('groq:llama-3.1-8b-instant')

result_sync = agent.run_sync(
    'What is the captial of Italy? Answer with just the city.',
    usage_limits=UsageLimits(response_tokens_limit=10)
)

print(result_sync.data)

print(result_sync.usage())


try:
    result_sync = agent.run_sync(
        'What is the capital of Italy?Answer with a paragraph.',
        usage_limits=UsageLimits(response_tokens_limit=10)
    )
except UsageLimitExceeded as e:
    print(e)


Rome
Usage(requests=1, request_tokens=49, response_tokens=3, total_tokens=52, details=None)
Exceeded the response_tokens_limit of 10 (response_tokens=150)


In [77]:
from typing_extensions import TypedDict

from pydantic_ai import Agent,ModelRetry
from pydantic_ai.exceptions import UsageLimitExceeded
from pydantic_ai.settings import UsageLimits

class NeverResultType(TypedDict):
    never_use_this:str

agent = Agent(
    'gemini-1.5-flash',
    result_type=NeverResultType,
    system_prompt="Any time you get a response,call the infinite_retry_tool"
                "to produce another response"
)


@agent.tool_plain(retries=5)
def infinite_retry_tool()->int:
    raise ModelRetry('Please try again')

try:
    result_sync = agent.run_sync(
        'Begin infinite retry loop! ', usage_limits=UsageLimits(request_limit=3)
    )

except UsageLimitExceeded as e:
    pprint(e)




The next request would exceed the request_limit of 3

In [79]:
from pydantic_ai import Agent

agent= Agent('gemini-1.5-flash')

result_sync= agent.run_sync(
    'What is the captial of Italy?' ,model_settings={'temperature':1.0}
)


print(result_sync.data)

Rome



In [81]:
from pydantic_ai import Agent

agent = Agent('gemini-1.5-flash')

result1 = agent.run_sync('Who was Albert Einstein.Tell in short?')

result2 = agent.run_sync(
    "What was his fost famous equation",
    message_history=result1.new_messages()
)

pprint(result1.data)
pprint(result2.data)

Albert Einstein (1879-1955) was a German-born theoretical physicist who developed the theory of relativity, one of 
the two pillars of modern physics.  His work is also known for its influence on the philosophy of science.  He is 
widely regarded as one of the most influential scientists of all time.

His most famous equation is **E=mc²**.

In [83]:
!pip install mypy 

  Using cached mypy-1.14.1-cp311-cp311-win_amd64.whl.metadata (2.2 kB)
   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.8 MB ? eta -:--:--
   ------- -------------------------------- 1.8/9.8 MB 6.7 MB/s eta 0:00:02
   -------------------- ------------------- 5.0/9.8 MB 10.1 MB/s eta 0:00:01
   ------------------------------ --------- 7.3/9.8 MB 10.3 MB/s eta 0:00:01
   ------------------------------------- -- 9.2/9.8 MB 10.0 MB/s eta 0:00:01
   ---------------------------------------- 9.8/9.8 MB 9.5 MB/s eta 0:00:00


In [84]:
%load_ext mypy


The mypy module is not an IPython extension.


Skipped

In [102]:
from pydantic import BaseModel

from pydantic_ai import Agent,RunContext,ModelRetry

from fake_database import DatabaseConn

class ChatResult(BaseModel):
    user_id:int
    message:str

agent= Agent(
        'groq:mixtral-8x7b-32768',
        deps_type=DatabaseConn,
        result_type=ChatResult,
        system_prompt="You are a message agent"
        "User the tool to get the name and if name not found"
        "Don't assume aything and do as said"
)

@agent.tool(retries=2)
def get_user_by_name(ctx:RunContext[DatabaseConn],name:str)->int:
    print(name)

    user_id = ctx.deps.users.get(name=name)
    if user_id is None:
        raise ModelRetry(
            f'No user found with name {name!r}.Retry'
        )
    
    return user_id

result = agent.run_sync(
    'Send a message to Tom  for coffee next week' ,deps = DatabaseConn()
)

pprint(result.data)





Jerry


UnexpectedModelBehavior: Tool exceeded max retries count of 2

In [104]:
from pydantic_ai import Agent, RunContext

roulette_agent = Agent(  
    'groq:llama-3.2-3b-preview',
    deps_type=int,
    result_type=bool,
    system_prompt=(
        'Use the `roulette_wheel` function to see if the '
        'customer has won based on the number they provide.'
    ),
)


@roulette_agent.tool
async def roulette_wheel(ctx: RunContext[int], square: int) -> str:  
    """check if the square is a winner"""
    return 'winner' if square == ctx.deps else 'loser'


# Run the agent
success_number = 18  
result = roulette_agent.run_sync('Put my money on square eighteen', deps=success_number)
print(result.data)  
#> True

result = roulette_agent.run_sync('I bet five is the winner', deps=success_number)
print(result.data)



True


BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': 'roulette_wheel>{square: 5}</function>'}}